⚡ Intermédiaire | ⏱ 45 min | 🔑 Concepts : @dataclass, field(), Enum, DTO, config

# 7. Dataclasses et Enums

## Objectifs

- Comprendre le problème du boilerplate dans les classes
- Maîtriser le décorateur @dataclass et ses options
- Utiliser field() pour personnaliser les attributs
- Créer des énumérations avec Enum
- Appliquer dataclasses et enums à des cas concrets

## Prérequis

- Classes et POO
- Décorateurs (bases)
- Type hints

## 1. Le problème du boilerplate

Les classes Python nécessitent souvent beaucoup de code répétitif.

In [ ]:
# Classe classique avec beaucoup de boilerplate
class PersonneClassique:
    def __init__(self, nom: str, age: int, email: str):
        self.nom = nom
        self.age = age
        self.email = email
    
    def __repr__(self):
        return f"PersonneClassique(nom={self.nom!r}, age={self.age!r}, email={self.email!r})"
    
    def __eq__(self, other):
        if not isinstance(other, PersonneClassique):
            return NotImplemented
        return (self.nom, self.age, self.email) == (other.nom, other.age, other.email)
    
    def __hash__(self):
        return hash((self.nom, self.age, self.email))

# Beaucoup de code pour une simple classe de données!
p1 = PersonneClassique("Alice", 30, "alice@example.com")
p2 = PersonneClassique("Alice", 30, "alice@example.com")

print(f"Personne: {p1}")
print(f"p1 == p2: {p1 == p2}")
print(f"Lignes de code: ~20")

## 2. Introduction aux dataclasses

Le décorateur `@dataclass` génère automatiquement les méthodes communes.

In [ ]:
from dataclasses import dataclass

# Même résultat avec beaucoup moins de code!
@dataclass
class Personne:
    nom: str
    age: int
    email: str

# Méthodes générées automatiquement:
# - __init__
# - __repr__
# - __eq__

p1 = Personne("Alice", 30, "alice@example.com")
p2 = Personne("Alice", 30, "alice@example.com")
p3 = Personne("Bob", 25, "bob@example.com")

print(f"Personne: {p1}")
print(f"p1 == p2: {p1 == p2}")
print(f"p1 == p3: {p1 == p3}")
print(f"\nAttributs accessibles:")
print(f"  Nom: {p1.nom}")
print(f"  Age: {p1.age}")
print(f"  Email: {p1.email}")

## 3. Valeurs par défaut

Les dataclasses supportent les valeurs par défaut.

In [ ]:
from dataclasses import dataclass
from typing import Optional

@dataclass
class Config:
    host: str = "localhost"
    port: int = 8080
    debug: bool = False
    ssl_enabled: bool = False
    timeout: Optional[int] = None

# Différentes façons de créer des instances
config1 = Config()
print(f"Config par défaut: {config1}")

config2 = Config(host="api.example.com")
print(f"\nConfig personnalisée: {config2}")

config3 = Config(
    host="prod.example.com",
    port=443,
    ssl_enabled=True,
    timeout=30
)
print(f"\nConfig production: {config3}")

## 4. field() pour personnaliser les attributs

`field()` permet de contrôler le comportement des attributs.

In [ ]:
from dataclasses import dataclass, field
from typing import List
import time

@dataclass
class Article:
    titre: str
    contenu: str
    # default_factory pour les valeurs mutables
    tags: List[str] = field(default_factory=list)
    # Champ calculé à la création
    created_at: float = field(default_factory=time.time)
    # Champ non affiché dans __repr__
    _internal_id: int = field(default=0, repr=False)
    # Champ non comparé dans __eq__
    views: int = field(default=0, compare=False)

# Tests
article1 = Article(
    titre="Python Tips",
    contenu="Voici des astuces Python...",
    tags=["python", "tips"]
)

article2 = Article(
    titre="Python Tips",
    contenu="Voici des astuces Python...",
    tags=["python", "tips"],
    views=100  # Différent, mais ignoré dans __eq__
)

print(f"Article 1: {article1}")
print(f"\nArticle 2: {article2}")
print(f"\nSont égaux? {article1 == article2}")  # True car views n'est pas comparé
print(f"Views différentes? {article1.views != article2.views}")  # True

# Vérifier que les listes sont différentes instances
article3 = Article(titre="Test", contenu="...")
article4 = Article(titre="Test", contenu="...")
article3.tags.append("tag1")
print(f"\nTags article 3: {article3.tags}")
print(f"Tags article 4: {article4.tags}")  # Liste vide, pas partagée!

## 5. __post_init__ : logique après initialisation

`__post_init__` est appelé après `__init__` pour ajouter de la logique.

In [ ]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Produit:
    nom: str
    prix_ht: float
    tva: float = 0.20
    prix_ttc: float = field(init=False)  # Calculé, pas dans __init__
    
    def __post_init__(self):
        # Calculer le prix TTC
        self.prix_ttc = self.prix_ht * (1 + self.tva)
        # Validation
        if self.prix_ht < 0:
            raise ValueError("Le prix ne peut pas être négatif")

@dataclass
class Commande:
    client: str
    produits: List[Produit] = field(default_factory=list)
    total: float = field(init=False, repr=True)
    
    def __post_init__(self):
        self.total = sum(p.prix_ttc for p in self.produits)

# Tests
produit1 = Produit("Clavier", 50.0)
produit2 = Produit("Souris", 30.0)
produit3 = Produit("Écran", 200.0, tva=0.055)  # TVA réduite

print(f"Produit 1: {produit1}")
print(f"  Prix HT: {produit1.prix_ht}€")
print(f"  Prix TTC: {produit1.prix_ttc:.2f}€")

commande = Commande(
    client="Alice",
    produits=[produit1, produit2, produit3]
)

print(f"\nCommande: {commande}")
print(f"Total commande: {commande.total:.2f}€")

# Validation
try:
    produit_invalide = Produit("Test", -10.0)
except ValueError as e:
    print(f"\nErreur: {e}")

## 6. Options de @dataclass

Le décorateur accepte plusieurs paramètres pour personnaliser le comportement.

In [ ]:
from dataclasses import dataclass

# frozen=True : dataclass immutable
@dataclass(frozen=True)
class Point:
    x: float
    y: float

p = Point(3.0, 4.0)
print(f"Point: {p}")

# Tentative de modification
try:
    p.x = 5.0  # Erreur!
except Exception as e:
    print(f"Erreur (immutable): {type(e).__name__}")

# frozen permet d'utiliser dans un set ou dict
points = {Point(1, 2), Point(3, 4), Point(1, 2)}  # Dédupliqué
print(f"\nPoints uniques: {points}")

# order=True : ajoute __lt__, __le__, __gt__, __ge__
@dataclass(order=True)
class Version:
    major: int
    minor: int
    patch: int = 0

versions = [
    Version(2, 0, 1),
    Version(1, 5, 0),
    Version(2, 1, 0),
    Version(1, 5, 2),
]

print(f"\nVersions non triées: {versions}")
print(f"Versions triées: {sorted(versions)}")

# Comparaisons
v1 = Version(1, 0, 0)
v2 = Version(2, 0, 0)
print(f"\nv1 < v2: {v1 < v2}")
print(f"v1 > v2: {v1 > v2}")

## 7. slots=True : optimisation mémoire (Python 3.10+)

`slots=True` réduit la consommation mémoire.

In [ ]:
from dataclasses import dataclass
import sys

# Sans slots
@dataclass
class PersonneSansSlots:
    nom: str
    age: int
    email: str

# Avec slots (Python 3.10+)
# @dataclass(slots=True)
# class PersonneAvecSlots:
#     nom: str
#     age: int
#     email: str

# Comparaison mémoire
p1 = PersonneSansSlots("Alice", 30, "alice@example.com")
# p2 = PersonneAvecSlots("Alice", 30, "alice@example.com")

print(f"Taille sans slots: {sys.getsizeof(p1)} bytes")
# print(f"Taille avec slots: {sys.getsizeof(p2)} bytes")

# Sans slots : __dict__ dynamique (flexible mais consomme plus)
print(f"\n__dict__ disponible: {hasattr(p1, '__dict__')}")
print(f"__dict__: {p1.__dict__}")

# On peut ajouter des attributs dynamiquement
p1.nouveau_attribut = "valeur"
print(f"Attribut ajouté: {p1.nouveau_attribut}")

# Avec slots : pas de __dict__, moins flexible mais plus efficace
# print(f"\n__dict__ disponible: {hasattr(p2, '__dict__')}")
# p2.nouveau_attribut = "valeur"  # Erreur!

## 8. Introduction aux Enums

Les énumérations permettent de définir un ensemble de constantes nommées.

In [ ]:
from enum import Enum

# Définir une énumération
class Statut(Enum):
    EN_ATTENTE = 1
    EN_COURS = 2
    TERMINE = 3
    ANNULE = 4

# Utilisation
statut_actuel = Statut.EN_COURS

print(f"Statut: {statut_actuel}")
print(f"Nom: {statut_actuel.name}")
print(f"Valeur: {statut_actuel.value}")

# Comparaison
print(f"\nEst en cours? {statut_actuel == Statut.EN_COURS}")
print(f"Est terminé? {statut_actuel == Statut.TERMINE}")

# Itération
print("\nTous les statuts:")
for statut in Statut:
    print(f"  {statut.name}: {statut.value}")

# Accès par valeur ou nom
statut_from_value = Statut(2)
statut_from_name = Statut['TERMINE']

print(f"\nDepuis valeur 2: {statut_from_value}")
print(f"Depuis nom 'TERMINE': {statut_from_name}")

## 9. Types d'Enums

Python propose plusieurs types d'énumérations.

In [ ]:
from enum import Enum, IntEnum, auto

# Enum classique
class Couleur(Enum):
    ROUGE = 1
    VERT = 2
    BLEU = 3

# IntEnum - se comporte comme un int
class Priorite(IntEnum):
    BASSE = 1
    NORMALE = 2
    HAUTE = 3
    CRITIQUE = 4

# auto() - valeurs automatiques
class Jour(Enum):
    LUNDI = auto()      # 1
    MARDI = auto()      # 2
    MERCREDI = auto()   # 3
    JEUDI = auto()      # 4
    VENDREDI = auto()   # 5
    SAMEDI = auto()     # 6
    DIMANCHE = auto()   # 7

# Tests
print("Couleurs:")
for c in Couleur:
    print(f"  {c.name}: {c.value}")

# IntEnum peut être comparé avec des int
p = Priorite.HAUTE
print(f"\nPriorité: {p}")
print(f"Priorité > 2: {p > 2}")
print(f"Priorité + 1: {p + 1}")

# auto() génère les valeurs
print("\nJours:")
for jour in Jour:
    print(f"  {jour.name}: {jour.value}")

## 10. StrEnum (Python 3.11+) et Enums avec méthodes

In [ ]:
from enum import Enum

# Enum avec méthodes
class HttpStatus(Enum):
    OK = 200
    CREATED = 201
    BAD_REQUEST = 400
    UNAUTHORIZED = 401
    FORBIDDEN = 403
    NOT_FOUND = 404
    SERVER_ERROR = 500
    
    def is_success(self) -> bool:
        """Vérifie si le status est un succès (2xx)."""
        return 200 <= self.value < 300
    
    def is_error(self) -> bool:
        """Vérifie si le status est une erreur (4xx ou 5xx)."""
        return self.value >= 400
    
    def is_client_error(self) -> bool:
        """Vérifie si c'est une erreur client (4xx)."""
        return 400 <= self.value < 500

# Tests
status = HttpStatus.OK
print(f"Status: {status.value} {status.name}")
print(f"Est un succès? {status.is_success()}")
print(f"Est une erreur? {status.is_error()}")

status_error = HttpStatus.NOT_FOUND
print(f"\nStatus: {status_error.value} {status_error.name}")
print(f"Est un succès? {status_error.is_success()}")
print(f"Est une erreur? {status_error.is_error()}")
print(f"Est erreur client? {status_error.is_client_error()}")

## 11. Cas concrets : DTO (Data Transfer Object)

Les dataclasses sont idéales pour les DTOs.

In [ ]:
from dataclasses import dataclass, field
from typing import List, Optional
from datetime import datetime

@dataclass
class UserDTO:
    """Data Transfer Object pour un utilisateur."""
    id: int
    username: str
    email: str
    first_name: Optional[str] = None
    last_name: Optional[str] = None
    is_active: bool = True
    created_at: datetime = field(default_factory=datetime.now)
    roles: List[str] = field(default_factory=list)
    
    @property
    def full_name(self) -> str:
        """Retourne le nom complet."""
        if self.first_name and self.last_name:
            return f"{self.first_name} {self.last_name}"
        return self.username
    
    def to_dict(self) -> dict:
        """Convertit en dictionnaire pour JSON."""
        return {
            'id': self.id,
            'username': self.username,
            'email': self.email,
            'first_name': self.first_name,
            'last_name': self.last_name,
            'full_name': self.full_name,
            'is_active': self.is_active,
            'created_at': self.created_at.isoformat(),
            'roles': self.roles,
        }

# Utilisation
user = UserDTO(
    id=1,
    username="alice",
    email="alice@example.com",
    first_name="Alice",
    last_name="Dupont",
    roles=["user", "admin"]
)

print(f"User: {user}")
print(f"\nNom complet: {user.full_name}")
print(f"\nDictionnaire:")
import json
print(json.dumps(user.to_dict(), indent=2))

## 12. Cas concrets : Configuration d'application

Utiliser dataclasses et enums pour la configuration.

In [ ]:
from dataclasses import dataclass
from enum import Enum
from typing import List

class Environment(Enum):
    DEVELOPMENT = "dev"
    STAGING = "staging"
    PRODUCTION = "prod"

class LogLevel(Enum):
    DEBUG = "DEBUG"
    INFO = "INFO"
    WARNING = "WARNING"
    ERROR = "ERROR"

@dataclass
class DatabaseConfig:
    host: str
    port: int
    database: str
    username: str
    password: str
    pool_size: int = 10
    
    def get_connection_string(self) -> str:
        return f"postgresql://{self.username}:{self.password}@{self.host}:{self.port}/{self.database}"

@dataclass
class AppConfig:
    app_name: str
    environment: Environment
    debug: bool
    log_level: LogLevel
    database: DatabaseConfig
    allowed_hosts: List[str] = None
    
    def __post_init__(self):
        if self.allowed_hosts is None:
            self.allowed_hosts = ["localhost"]
        
        # Validation
        if self.environment == Environment.PRODUCTION and self.debug:
            raise ValueError("Debug mode cannot be enabled in production")

# Configuration de développement
dev_config = AppConfig(
    app_name="MyApp",
    environment=Environment.DEVELOPMENT,
    debug=True,
    log_level=LogLevel.DEBUG,
    database=DatabaseConfig(
        host="localhost",
        port=5432,
        database="myapp_dev",
        username="dev_user",
        password="dev_password"
    )
)

print(f"Config: {dev_config.app_name}")
print(f"Environment: {dev_config.environment.value}")
print(f"Log level: {dev_config.log_level.value}")
print(f"DB Connection: {dev_config.database.get_connection_string()}")
print(f"Allowed hosts: {dev_config.allowed_hosts}")

# Tentative de config invalide
try:
    prod_config = AppConfig(
        app_name="MyApp",
        environment=Environment.PRODUCTION,
        debug=True,  # Invalide!
        log_level=LogLevel.ERROR,
        database=dev_config.database
    )
except ValueError as e:
    print(f"\nErreur de validation: {e}")

## 13. Dataclass vs namedtuple vs dict

Comparaison des différentes approches.

In [ ]:
from dataclasses import dataclass
from typing import NamedTuple

# 1. Dict - flexible mais pas de type safety
person_dict = {
    'nom': 'Alice',
    'age': 30,
    'email': 'alice@example.com'
}

# 2. NamedTuple - immutable, léger
class PersonTuple(NamedTuple):
    nom: str
    age: int
    email: str

person_tuple = PersonTuple('Alice', 30, 'alice@example.com')

# 3. Dataclass - flexible, mutable par défaut
@dataclass
class PersonDataclass:
    nom: str
    age: int
    email: str

person_dataclass = PersonDataclass('Alice', 30, 'alice@example.com')

# Comparaisons
print("=" * 60)
print("COMPARAISON DES APPROCHES")
print("=" * 60)

print("\n1. DICT")
print(f"   Accès: person_dict['nom'] = {person_dict['nom']}")
print(f"   Modification: OK")
person_dict['age'] = 31
print(f"   Nouvel âge: {person_dict['age']}")
print(f"   Type hints: ❌")
print(f"   IDE autocomplete: ❌")
print(f"   Immutable: ❌")

print("\n2. NAMEDTUPLE")
print(f"   Accès: person_tuple.nom = {person_tuple.nom}")
print(f"   Modification: ❌ (immutable)")
try:
    person_tuple.age = 31
except AttributeError:
    print("   Erreur: can't set attribute")
print(f"   Type hints: ✅")
print(f"   IDE autocomplete: ✅")
print(f"   Immutable: ✅")
print(f"   Léger en mémoire: ✅")

print("\n3. DATACLASS")
print(f"   Accès: person_dataclass.nom = {person_dataclass.nom}")
print(f"   Modification: ✅")
person_dataclass.age = 31
print(f"   Nouvel âge: {person_dataclass.age}")
print(f"   Type hints: ✅")
print(f"   IDE autocomplete: ✅")
print(f"   Immutable: ✅ (avec frozen=True)")
print(f"   Méthodes custom: ✅")
print(f"   __post_init__: ✅")

print("\n" + "=" * 60)
print("RECOMMANDATIONS:")
print("  - Dict: données simples, pas de structure fixe")
print("  - NamedTuple: petites structures immutables")
print("  - Dataclass: structures complexes, DTOs, configs")
print("=" * 60)

## Pièges courants

### 1. Mutable default values

In [ ]:
from dataclasses import dataclass, field
from typing import List

# PIÈGE: Ne JAMAIS utiliser de liste comme valeur par défaut
# @dataclass
# class TaskList:
#     tasks: List[str] = []  # ERREUR!

# SOLUTION: Utiliser field(default_factory=list)
@dataclass
class TaskList:
    tasks: List[str] = field(default_factory=list)

# Test
list1 = TaskList()
list2 = TaskList()

list1.tasks.append("Task 1")
list2.tasks.append("Task 2")

print(f"List 1: {list1.tasks}")  # ['Task 1']
print(f"List 2: {list2.tasks}")  # ['Task 2']
print(f"Listes différentes: {list1.tasks is not list2.tasks}")

### 2. Ordre des champs avec valeurs par défaut

In [ ]:
from dataclasses import dataclass

# PIÈGE: Champs sans défaut après champs avec défaut
# @dataclass
# class MauvaiseClasse:
#     nom: str = "default"
#     age: int  # ERREUR!

# SOLUTION 1: Mettre les champs sans défaut en premier
@dataclass
class BonneClasse:
    age: int  # Sans défaut en premier
    nom: str = "default"

# SOLUTION 2: Donner une valeur par défaut à tous
@dataclass
class AutreSolution:
    nom: str = "default"
    age: int = 0

# Tests
obj1 = BonneClasse(30)
obj2 = BonneClasse(30, "Alice")
print(f"Obj1: {obj1}")
print(f"Obj2: {obj2}")

### 3. Comparaison d'Enums

In [ ]:
from enum import Enum, IntEnum

class Statut(Enum):
    ACTIF = 1
    INACTIF = 2

class Priority(IntEnum):
    LOW = 1
    HIGH = 2

# PIÈGE: Enum ne se compare PAS avec les valeurs
statut = Statut.ACTIF
print(f"statut == 1: {statut == 1}")  # False!
print(f"statut.value == 1: {statut.value == 1}")  # True

# IntEnum se compare avec int
priority = Priority.LOW
print(f"\npriority == 1: {priority == 1}")  # True
print(f"priority < Priority.HIGH: {priority < Priority.HIGH}")  # True

# RECOMMANDATION: Toujours comparer avec les membres
print(f"\nstatut == Statut.ACTIF: {statut == Statut.ACTIF}")  # True - CORRECT

## Mini-exercices

### Exercice 1: Dataclass pour configuration

Créez une dataclass `ServerConfig` avec validation dans `__post_init__`.

In [ ]:
# À compléter
# Créer une dataclass avec:
# - host: str (défaut: "localhost")
# - port: int (défaut: 8080)
# - workers: int (défaut: 4)
# - max_connections: int (défaut: 100)
# Validation:
# - port entre 1 et 65535
# - workers >= 1
# - max_connections >= workers

# Test
# config = ServerConfig(port=8000, workers=8, max_connections=200)
# print(config)

### Solution Exercice 1

In [ ]:
from dataclasses import dataclass

@dataclass
class ServerConfig:
    host: str = "localhost"
    port: int = 8080
    workers: int = 4
    max_connections: int = 100
    
    def __post_init__(self):
        # Validation du port
        if not (1 <= self.port <= 65535):
            raise ValueError(f"Port doit être entre 1 et 65535, reçu {self.port}")
        
        # Validation des workers
        if self.workers < 1:
            raise ValueError(f"Workers doit être >= 1, reçu {self.workers}")
        
        # Validation des max_connections
        if self.max_connections < self.workers:
            raise ValueError(
                f"max_connections ({self.max_connections}) doit être >= workers ({self.workers})"
            )

# Tests valides
config1 = ServerConfig()
print(f"Config par défaut: {config1}")

config2 = ServerConfig(port=8000, workers=8, max_connections=200)
print(f"\nConfig personnalisée: {config2}")

# Tests invalides
try:
    ServerConfig(port=70000)
except ValueError as e:
    print(f"\nErreur port: {e}")

try:
    ServerConfig(workers=0)
except ValueError as e:
    print(f"Erreur workers: {e}")

try:
    ServerConfig(workers=10, max_connections=5)
except ValueError as e:
    print(f"Erreur max_connections: {e}")

### Exercice 2: Enum pour système de statuts

Créez un système de gestion de commandes avec Enum.

In [ ]:
# À compléter
# Créer un Enum OrderStatus avec:
# - PENDING, PROCESSING, SHIPPED, DELIVERED, CANCELLED
# Ajouter des méthodes:
# - can_cancel() -> bool (uniquement PENDING et PROCESSING)
# - is_final() -> bool (DELIVERED ou CANCELLED)

# Test
# status = OrderStatus.PROCESSING
# print(f"Peut annuler? {status.can_cancel()}")
# print(f"Est final? {status.is_final()}")

### Solution Exercice 2

In [ ]:
from enum import Enum, auto
from dataclasses import dataclass
from datetime import datetime

class OrderStatus(Enum):
    PENDING = auto()
    PROCESSING = auto()
    SHIPPED = auto()
    DELIVERED = auto()
    CANCELLED = auto()
    
    def can_cancel(self) -> bool:
        """Vérifie si la commande peut être annulée."""
        return self in (OrderStatus.PENDING, OrderStatus.PROCESSING)
    
    def is_final(self) -> bool:
        """Vérifie si le statut est final."""
        return self in (OrderStatus.DELIVERED, OrderStatus.CANCELLED)
    
    def next_status(self) -> 'OrderStatus | None':
        """Retourne le prochain statut possible."""
        transitions = {
            OrderStatus.PENDING: OrderStatus.PROCESSING,
            OrderStatus.PROCESSING: OrderStatus.SHIPPED,
            OrderStatus.SHIPPED: OrderStatus.DELIVERED,
        }
        return transitions.get(self)

@dataclass
class Order:
    id: int
    customer: str
    total: float
    status: OrderStatus = OrderStatus.PENDING
    created_at: datetime = None
    
    def __post_init__(self):
        if self.created_at is None:
            self.created_at = datetime.now()
    
    def cancel(self) -> bool:
        """Annule la commande si possible."""
        if self.status.can_cancel():
            self.status = OrderStatus.CANCELLED
            return True
        return False
    
    def advance(self) -> bool:
        """Avance au prochain statut si possible."""
        next_status = self.status.next_status()
        if next_status:
            self.status = next_status
            return True
        return False

# Tests
order = Order(id=1, customer="Alice", total=99.99)
print(f"Commande initiale: {order.status.name}")
print(f"Peut annuler? {order.status.can_cancel()}")
print(f"Est final? {order.status.is_final()}")

# Progression de la commande
print("\nProgression:")
while order.advance():
    print(f"  Nouveau statut: {order.status.name}")

print(f"\nStatut final? {order.status.is_final()}")

# Test d'annulation
order2 = Order(id=2, customer="Bob", total=49.99)
print(f"\nCommande 2: {order2.status.name}")
order2.advance()
print(f"Après advance: {order2.status.name}")
print(f"Annulation: {order2.cancel()}")
print(f"Statut final: {order2.status.name}")

### Exercice 3: DTO pour pipeline de données

Créez des DTOs pour un pipeline de transformation de données.

In [ ]:
# À compléter
# Créer:
# 1. PipelineStatus (Enum): IDLE, RUNNING, SUCCESS, FAILED
# 2. PipelineMetrics (dataclass): records_processed, duration_seconds, errors
# 3. PipelineRun (dataclass): run_id, status, metrics, started_at, finished_at

# Test
# run = PipelineRun(...)
# print(run)

### Solution Exercice 3

In [ ]:
from dataclasses import dataclass, field
from enum import Enum, auto
from datetime import datetime
from typing import Optional, List

class PipelineStatus(Enum):
    IDLE = auto()
    RUNNING = auto()
    SUCCESS = auto()
    FAILED = auto()
    
    def is_complete(self) -> bool:
        return self in (PipelineStatus.SUCCESS, PipelineStatus.FAILED)

@dataclass
class PipelineMetrics:
    records_processed: int = 0
    duration_seconds: float = 0.0
    errors: List[str] = field(default_factory=list)
    
    @property
    def throughput(self) -> float:
        """Records par seconde."""
        if self.duration_seconds > 0:
            return self.records_processed / self.duration_seconds
        return 0.0
    
    @property
    def has_errors(self) -> bool:
        return len(self.errors) > 0

@dataclass
class PipelineRun:
    run_id: str
    pipeline_name: str
    status: PipelineStatus = PipelineStatus.IDLE
    metrics: PipelineMetrics = field(default_factory=PipelineMetrics)
    started_at: Optional[datetime] = None
    finished_at: Optional[datetime] = None
    
    def start(self) -> None:
        """Démarre le pipeline."""
        self.status = PipelineStatus.RUNNING
        self.started_at = datetime.now()
    
    def complete(self, success: bool = True) -> None:
        """Termine le pipeline."""
        self.finished_at = datetime.now()
        self.status = PipelineStatus.SUCCESS if success else PipelineStatus.FAILED
        
        if self.started_at and self.finished_at:
            duration = (self.finished_at - self.started_at).total_seconds()
            self.metrics.duration_seconds = duration
    
    def to_dict(self) -> dict:
        """Convertit en dictionnaire."""
        return {
            'run_id': self.run_id,
            'pipeline_name': self.pipeline_name,
            'status': self.status.name,
            'records_processed': self.metrics.records_processed,
            'duration_seconds': self.metrics.duration_seconds,
            'throughput': self.metrics.throughput,
            'errors': self.metrics.errors,
            'started_at': self.started_at.isoformat() if self.started_at else None,
            'finished_at': self.finished_at.isoformat() if self.finished_at else None,
        }

# Simulation d'un pipeline
import time

run = PipelineRun(
    run_id="run_001",
    pipeline_name="data_ingestion"
)

print(f"Pipeline créé: {run.status.name}")

# Démarrage
run.start()
print(f"\nPipeline démarré: {run.status.name}")

# Simulation de traitement
time.sleep(0.1)
run.metrics.records_processed = 1000

# Fin du pipeline
run.complete(success=True)
print(f"\nPipeline terminé: {run.status.name}")
print(f"Records traités: {run.metrics.records_processed}")
print(f"Durée: {run.metrics.duration_seconds:.2f}s")
print(f"Throughput: {run.metrics.throughput:.0f} records/s")

print("\nDictionnaire:")
import json
print(json.dumps(run.to_dict(), indent=2))